## 图像卷积 

### 互相关运算 

In [35]:
import torch
from torch import nn
from d2l import torch as d2l

#cross-correlation
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

In [36]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])

In [37]:
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

### 卷积层 

In [38]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [39]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [40]:
K = torch.tensor([[-1.0, 1.0]])

In [41]:
Y = corr2d(X, K)
Y

tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

In [48]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(100):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 3.956
epoch 4, loss 0.711
epoch 6, loss 0.138
epoch 8, loss 0.031
epoch 10, loss 0.008
epoch 12, loss 0.003
epoch 14, loss 0.001
epoch 16, loss 0.000
epoch 18, loss 0.000
epoch 20, loss 0.000
epoch 22, loss 0.000
epoch 24, loss 0.000
epoch 26, loss 0.000
epoch 28, loss 0.000
epoch 30, loss 0.000
epoch 32, loss 0.000
epoch 34, loss 0.000
epoch 36, loss 0.000
epoch 38, loss 0.000
epoch 40, loss 0.000
epoch 42, loss 0.000
epoch 44, loss 0.000
epoch 46, loss 0.000
epoch 48, loss 0.000
epoch 50, loss 0.000
epoch 52, loss 0.000
epoch 54, loss 0.000
epoch 56, loss 0.000
epoch 58, loss 0.000
epoch 60, loss 0.000
epoch 62, loss 0.000
epoch 64, loss 0.000
epoch 66, loss 0.000
epoch 68, loss 0.000
epoch 70, loss 0.000
epoch 72, loss 0.000
epoch 74, loss 0.000
epoch 76, loss 0.000
epoch 78, loss 0.000
epoch 80, loss 0.000
epoch 82, loss 0.000
epoch 84, loss 0.000
epoch 86, loss 0.000
epoch 88, loss 0.000
epoch 90, loss 0.000
epoch 92, loss 0.000
epoch 94, loss 0.000
epoch 96, loss 0.

In [49]:
conv2d.weight.data.reshape((1, 2))

tensor([[-1.0000,  1.0000]])

### 填充


In [56]:
def comp_conv2d(conv2d, X):
    X = X.reshape((1,1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])

#上下左右各填充1个像素
conv2d = nn.Conv2d(1,1,kernel_size=3, padding=1)
X = torch.rand(size=(8,8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [60]:
#将高度和宽度的步幅
X = torch.rand(size=(8,8))
conv2d = nn.Conv2d(1,1,kernel_size=(5,3), padding=(2,1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

### 多输入和输出通道

In [64]:
def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [65]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 3, 1, 2])

In [66]:
corr2d_multi_in_out(X, K)

IndexError: tuple index out of range